In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

c:\Users\misso\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
! pip install git+https://github.com/tensorflow/examples.git


  Cloning https://github.com/tensorflow/examples.git to c:\users\misso\appdata\local\temp\pip-req-build-68ubitpn
  Resolved https://github.com/tensorflow/examples.git to commit fff4bcda7201645a1efaea4534403daf5fc03d42
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tensorflow-examples: filename=tensorflow_examples-0.1703207612.1461250479831370929614362828255168868146460245314-py3-none-any.whl size=303791 sha256=d8b757b439d0956f7712e1aa27389a86ec89f49c3b5a1ece3310d6ba1df70218
  Stored in directory: C:\Users\misso\AppData\Local\Temp\pip-ephem-wheel-cache-pkkt3qmt\wheels\91\9b\e8\6ae2ecc930bd726c578e35b313e987a687bc5ce03c3a42c2d5
Successfully built tensorflow-examples


  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/examples.git 'C:\Users\misso\AppData\Local\Temp\pip-req-build-68ubitpn'


In [3]:
# can translate images from one domain to another
# (e.g., grayscale to color, maps to aerial photos).
from tensorflow_examples.models.pix2pix import pix2pix

#  plots and visualizations, including images.
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [4]:
# The with_info=True flag tells TensorFlow Datasets to 
# also return information about the dataset, 
# such as its description, splits, and feature descriptions.
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  50%|█████     | 1/2 [01:15<00:19, 19.16s/ url]



In [ ]:
def normalize(input_image, input_mask): # image tensor, mask tensor
  input_image = tf.cast(input_image, tf.float32) / 255.0
  # This is likely because masks commonly use 0 to represent
  # invalid pixels and 1 for valid ones.
  # Subtracting 1 converts the mask values to -1 for invalid
  # and 0 for valid.
  input_mask -= 1
  return input_image, input_mask

In [ ]:
#  Resizes and normalizes the images and masks.
def load_image(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(
    datapoint['segmentation_mask'],
    (128, 128),
    # interpolation to fill in missing pixels during resizing 
    method = tf.image.ResizeMethod.NEAREST_NEIGHBOR,
  )

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [ ]:
# retrieves the number of examples in the training split 
# of the dataset you loaded using tfds.load
TRAIN_LENGTH = info.splits['train'].num_examples
# determines how many data points are processed
# together during each training step
BATCH_SIZE = 64
# sets the buffer size for shuffling the dataset
# prevent the model from overfitting to the training data order
BUFFER_SIZE = 1000
# calculates the number of steps required to iterate over the
# entire training dataset once (one epoch)
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
# extracts the training and testing splits
# Applies the load_image function to each datapoint in the split.
train_images = dataset['train'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_images = dataset['test'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
class Augment(tf.keras.layers.Layer): # allowing it to be integrated into a Keras model.

  # two tf.keras.layers.RandomFlip layers are created in init()
  def __init__(self, seed=42):
    super().__init__()
    # both use the same seed, so they'll make the same random changes.
    # Flips input images horizontally with the specified seed
    self.augment_inputs = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
    # Flips segmentation masks horizontally
    self.augment_labels = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)

  # It returns the augmented inputs and labels as a tuple.
  def call(self, inputs, labels):
    inputs = self.augment_inputs(inputs)
    labels = self.augment_labels(labels)
    return inputs, labels

In [ ]:
train_batches = (
    train_images
    # caches the entire training dataset
    # in memory, reducing I/O overhead for subsequent training epochs.
    .cache()
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .repeat()
    .map(Augment())
    # prefetches the next batch of data 
    # while the current batch is being processed by the model
    .prefetch(buffer_size=tf.data.AUTOTUNE))

# batches the testing data into groups of BATCH_SIZE
# examples for efficient evaluation
test_batches = test_images.batch(BATCH_SIZE)

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15)) # 15x15 inches

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    #  converts the image tensor from TensorFlow format to a format suitable for display
    plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
for images, masks in train_batches.take(2):
  sample_image, sample_mask = images[0], masks[0]
  display([sample_image, sample_mask])

In [ ]:
# include_top=False to exclude the classification head of the pre-trained model,
# focusing only on extracting features.
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Use the activations of these layers
# These layers represent different stages of the model with decreasing spatial
# resolution (from 64x64 to 4x4) but increasing semantic complexity
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]

base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

# These outputs represent different levels of features extracted from the input image..
# Which are the activations of the specified layers in the base model,
# retrieved using base_model.get_layer(name).output.
down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

down_stack.trainable = False

In [ ]:
# upsamples and combines features to reconstruct the segmentation mask 
# in a coarse-to-fine manner.
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def unet_model(output_channels:int):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])

  # Downsampling through the model
  skips = down_stack(inputs) # which extracts features at different resolutions.
  # It takes the last element of skips (highest resolution features)
  x = skips[-1]

  # It reverses the remaining elements of skips (lower resolution features),
  # to have them in ascending order of resolution
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x) # It upsamples the current using corresponding up_stack layer
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      filters=output_channels, kernel_size=3, strides=2,
      padding='same')  #64x64 -> 128x128

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
OUTPUT_CLASSES = 3

model = unet_model(output_channels=OUTPUT_CLASSES)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.math.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
show_predictions()

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [ ]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_batches, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_batches,
                          callbacks=[DisplayCallback()])

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.figure()
plt.plot(model_history.epoch, loss, 'r', label='Training loss')
plt.plot(model_history.epoch, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
show_predictions(test_batches, 3)

In [ ]:
try:
  model_history = model.fit(train_batches, epochs=EPOCHS,
                            steps_per_epoch=STEPS_PER_EPOCH,
                            class_weight = {0:2.0, 1:2.0, 2:1.0})
  assert False
except Exception as e:
  print(f"Expected {type(e).__name__}: {e}")

In [ ]:
label = [0,0]
prediction = [[-3., 0], [-3, 0]] 
sample_weight = [1, 10] 

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                               reduction=tf.keras.losses.Reduction.NONE)
loss(label, prediction, sample_weight).numpy()

In [ ]:
def add_sample_weights(image, label):
  # The weights for each class, with the constraint that:
  #     sum(class_weights) == 1.0
  class_weights = tf.constant([2.0, 2.0, 1.0])
  class_weights = class_weights/tf.reduce_sum(class_weights)

  # Create an image of `sample_weights` by using the label at each pixel as an 
  # index into the `class weights` .
  sample_weights = tf.gather(class_weights, indices=tf.cast(label, tf.int32))

  return image, label, sample_weights

In [ ]:
train_batches.map(add_sample_weights).element_spec

In [ ]:
weighted_model = unet_model(OUTPUT_CLASSES)
weighted_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
weighted_model.fit(
    train_batches.map(add_sample_weights),
    epochs=1,
    steps_per_epoch=10)